# Universal Model Routing Experiment

**Paper**: [Universal Model Routing for Efficient LLM Inference](https://arxiv.org/pdf/2502.08773)  
**Authors**: Jitkrittum et al. (2025)

## Core Concept
Route queries to different LLMs using **cluster-based error profiles** that work with new unseen models without retraining.

**Key Innovation**: Ψ(m) vectors - error rates per question cluster for each model.

In [1]:
# Setup
!pip install -q openai scikit-learn numpy sentence-transformers datasets pandas pydantic matplotlib groq

import os
import json
import pickle
import numpy as np
import time
from typing import List, Dict, Optional, Literal
from collections import defaultdict
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel
import matplotlib.pyplot as plt
from datasets import load_dataset

print("✅ Setup complete")

✅ Setup complete


In [2]:
# Add your API keys
API_KEYS = {
    'openai': 'your-openai-key-here',
    'groq': 'your-groq-key-here'
}

MODELS = [
    {'name': 'gpt-4o', 'provider': 'openai', 'cost': 6.25},
    {'name': 'gpt-4.1', 'provider': 'openai', 'cost': 5.00},
    {'name': 'gpt-4o-mini', 'provider': 'openai', 'cost': 0.375},
    {'name': 'meta-llama/llama-guard-4-12b', 'provider': 'groq', 'cost': 0.20},
    {'name': 'llama-3.1-8b-instant', 'provider': 'groq', 'cost': 0.065}
]

print(f"✅ {len(MODELS)} models")
print(f"📊 Cost: ${min(m['cost'] for m in MODELS)} - ${max(m['cost'] for m in MODELS)}")

✅ 5 models
📊 Cost: $0.065 - $6.25


## Validation Set + Clustering

**Paper Method**: MMLU validation set, K-means clustering on question embeddings

In [3]:
# Load MMLU validation
dataset = load_dataset("cais/mmlu", "all", split="validation")
dataset = dataset.shuffle(seed=42).select(range(200))

# Format for MCQ
VALIDATION_SET = []
for i, item in enumerate(dataset):
    prompt = f"{item['question']}\nA) {item['choices'][0]}\nB) {item['choices'][1]}\nC) {item['choices'][2]}\nD) {item['choices'][3]}\nAnswer:"
    VALIDATION_SET.append({
        'prompt': prompt,
        'answer': ['A', 'B', 'C', 'D'][item['answer']],
        'question': item['question']
    })

print(f"✅ {len(VALIDATION_SET)} validation examples")

✅ 200 validation examples


In [4]:
# Cluster questions
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode([item['question'] for item in VALIDATION_SET])

K = 10
kmeans = KMeans(n_clusters=K, random_state=42)
clusters = kmeans.fit_predict(embeddings)

# Group by cluster
validation_clusters = defaultdict(list)
for i, cluster_id in enumerate(clusters):
    validation_clusters[cluster_id].append(VALIDATION_SET[i])

print(f"✅ {K} clusters created")
for k in range(K):
    print(f"  Cluster {k}: {len(validation_clusters[k])} examples")

/usr/local/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ 10 clusters created
  Cluster 0: 25 examples
  Cluster 1: 23 examples
  Cluster 2: 11 examples
  Cluster 3: 25 examples
  Cluster 4: 26 examples
  Cluster 5: 8 examples
  Cluster 6: 20 examples
  Cluster 7: 25 examples
  Cluster 8: 13 examples
  Cluster 9: 24 examples


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/anaconda3/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
